In [1]:
"""
Author: Ngo Van Uc
Date:
Contact: ngovanuc.1508@gmail.com
"""

'\nAuthor: Ngo Van Uc\nDate:\nContact: ngovanuc.1508@gmail.com\n'

In [2]:
from google.colab import drive
drive.mount('/content/MyDrive')

Mounted at /content/MyDrive


In [3]:
import warnings
warnings.filterwarnings('ignore')
from IPython.display import clear_output

import pandas as pd
import numpy as np

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

# 01. Preprocessing

In [4]:
train_path = '/content/MyDrive/MyDrive/Colab Notebooks/Aspect_Based_Sentiment_Analysis/Dataset/Train.csv'
test_path = '/content/MyDrive/MyDrive/Colab Notebooks/Aspect_Based_Sentiment_Analysis/Dataset/Test.csv'
dev_path = '/content/MyDrive/MyDrive/Colab Notebooks/Aspect_Based_Sentiment_Analysis/Dataset/Dev.csv'

train_data = pd.read_csv(train_path)
train_data.head()

,index,comment,n_star,date_time,label
0,0,Mới mua máy này Tại thegioididong thốt nốt cảm...,5,2 tuần trước,{CAMERA#Positive};{FEATURES#Positive};{BATTERY...
1,1,Pin kém còn lại miễn chê mua 8/3/2019 tình trạ...,5,14/09/2019,{BATTERY#Negative};{GENERAL#Positive};{OTHERS};
2,2,Sao lúc gọi điện thoại màn hình bị chấm nhỏ nh...,3,17/08/2020,{FEATURES#Negative};
3,3,"Mọi người cập nhật phần mềm lại , nó sẽ bớt tố...",3,29/02/2020,{FEATURES#Negative};{BATTERY#Neutral};{GENERAL...
4,4,"Mới mua Sài được 1 tháng thấy pin rất trâu, Sà...",5,4/6/2020,{BATTERY#Positive};{PERFORMANCE#Positive};{SER...


In [5]:
class_to_id = {"SCREEN": 0, "CAMERA": 1, "FEATURES": 2, "BATTERY": 3, "PERFORMANCE": 4, "STORAGE": 5, "DESIGN": 6, "PRICE": 7, "GENERAL": 8, "SER&ACC": 9}
label_to_id = {"Positive": 0, "Negative": 1, "Neutral": 2, "None": 3}

In [6]:
def get_label_dict(sentences_label):
    label_dict = {}
    labels = sentences_label.split(';')[:-1]
    if len(labels) == 1 and labels[-1][1:-1] == "OTHERS":
        return None
    else:
        if labels[-1][1:-1] == "OTHERS":
            for l in labels[:-1]:
                class_name, sentiment = l[1:-1].split('#')
                label_dict[class_name] = sentiment
            return label_dict
        else:
            for l in labels:
                class_name, sentiment = l[1:-1].split('#')
                label_dict[class_name] = sentiment
            return label_dict

In [7]:
sentences = train_data['comment'].tolist()
label_comment = train_data['label'].tolist()
others_label = [[0, 0, 0, 1]] * 10

train_set = []

for sentence, sentence_label in zip(sentences, label_comment):
    label_dict = get_label_dict(sentence_label)
    labels_ = []
    if label_dict:
        for aspect in list(class_to_id.keys()):
            train_label = [0] * 4
            if aspect in label_dict.keys():
                label_id = label_to_id[label_dict[aspect]]
                train_label[label_id] =  1
                labels_.append(train_label)
            else:
                labels_.append([0, 0, 0, 1])
        train_set.append(
            {'sentence': sentence, 'labels': labels_}
        )
    else:
        train_set.append(
            {'sentence': sentence, 'labels': others_label}
        )
print(train_set[0]['sentence'])
print(train_set[0]['labels'])

Mới mua máy này Tại thegioididong thốt nốt cảm thấy ok bin trâu chụp ảnh đẹp loa nghe to bắt wf khỏe sóng ổn định, giá thành vừa với túi tiền, nhân viên tư vấn nhiệt tình
[[0, 0, 0, 1], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 1], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0]]


In [8]:
for i, data in enumerate(train_set):
    sentence, label = data.items()
    input = np.array(label[-1])
    clear_output()
    if input.shape == (10, 4):
        print("pass")
    else:
        print(i)

pass


# 02. Build data loader

In [9]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer


class AspectSentimentDataset(Dataset):
    def __init__(self, data, tokenizer, max_len):
        self.data = data
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        sentence = self.data[index]['sentence']
        labels = self.data[index]['labels']

        encoding = self.tokenizer.encode_plus(
            sentence,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            pad_to_max_length=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        return {
            'sentence_text': sentence,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(labels, dtype=torch.float)
        }

In [10]:
tokenizer = BertTokenizer.from_pretrained('google-bert/bert-base-multilingual-cased')
dataset = AspectSentimentDataset(train_set, tokenizer, max_len=128)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

In [11]:
dataset.__len__()

7786

# 03. Build GRU model

In [12]:
import torch.nn as nn
from transformers import BertModel


class AspectSentimentModel(nn.Module):
    def __init__(self, n_classes, n_labels_per_class):
        super(AspectSentimentModel, self).__init__()
        self.bert = BertModel.from_pretrained('google-bert/bert-base-multilingual-cased')
        self.gru = nn.GRU(768, 256, batch_first=True, bidirectional=True)
        self.classifier = nn.Linear(256*2, n_classes * n_labels_per_class)

    def forward(self, input_ids, attention_mask):
        bert_outputs = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        gru_outputs, _ = self.gru(bert_outputs.last_hidden_state)
        gru_outputs = gru_outputs[:, -1, :]
        output = self.classifier(gru_outputs)
        return output.view(-1, 10, 4)

model = AspectSentimentModel(n_classes=10, n_labels_per_class=4)

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

In [14]:
import torch.optim as optim


def train_epoch(model, data_loader, loss_fn, optimizer, device):
    model = model.train()
    losses = []
    correct_predictions = 0

    for data in data_loader:
        input_ids = data['input_ids'].to(device)
        attention_mask = data['attention_mask'].to(device)
        labels = data['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)

        loss = loss_fn(outputs, labels)
        losses.append(loss.item())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    return np.mean(losses)


EPOCHS = 40
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
training = True


optimizer = optim.Adam(model.parameters(), lr=2e-5)
loss_fn = nn.BCEWithLogitsLoss()

if training == True:
    for epoch in range(EPOCHS):
        train_loss = train_epoch(model, dataloader, loss_fn, optimizer, device)
        print(f'Epoch {epoch + 1}/{EPOCHS}, Train loss: {train_loss}')
        if epoch % 5 == 0:
            torch.save(model.state_dict(), f'/content/MyDrive/MyDrive/Colab Notebooks/Aspect_Based_Sentiment_Analysis/Models/ABSA_using_GRU_model{epoch}.pth')

torch.save(model.state_dict(), f'/content/MyDrive/MyDrive/Colab Notebooks/Aspect_Based_Sentiment_Analysis/Models/ABSA_using_GRU_model{40}.pth')

Epoch 1/40, Train loss: 0.23072011462839231
Epoch 2/40, Train loss: 0.18701695450399936
Epoch 3/40, Train loss: 0.15728996366690806
Epoch 4/40, Train loss: 0.13490936134079398
Epoch 5/40, Train loss: 0.11640188646420561
Epoch 6/40, Train loss: 0.10072557250271098
Epoch 7/40, Train loss: 0.08754791470064764
Epoch 8/40, Train loss: 0.07631127020722787
Epoch 9/40, Train loss: 0.06634403382345759
Epoch 10/40, Train loss: 0.05877155634656824
Epoch 11/40, Train loss: 0.05144381649730999
Epoch 12/40, Train loss: 0.04509158282171652
Epoch 13/40, Train loss: 0.041310350527294606
Epoch 14/40, Train loss: 0.03630267967540748
Epoch 15/40, Train loss: 0.031870108694450075
Epoch 16/40, Train loss: 0.02851494640356706
Epoch 17/40, Train loss: 0.025057866203635013
Epoch 18/40, Train loss: 0.02266153636593104
Epoch 19/40, Train loss: 0.020191683438730863
Epoch 20/40, Train loss: 0.019690822266822477
Epoch 21/40, Train loss: 0.017744007830181038
Epoch 22/40, Train loss: 0.018211091171521733
Epoch 23/40,

In [16]:
# path_save_model = 'E:/IAD/Intern/11_aspectsAnalysis/models'
# torch.save(model.state_dict(), path_save_model+'/ABSA_using_GRU_model.pth')
# torch.save(model.state_dict(), f'/content/MyDrive/MyDrive/Colab Notebooks/Aspect_Based_Sentiment_Analysis/models/ABSA_using_GRU_model{40}.pth')

In [17]:
# model = AspectSentimentModel(n_classes=10, n_labels_per_class=4)
# model.load_state_dict(torch.load(path_save_model+'/ABSA_using_GRU_model.pth'))
# model.eval()

# 04. Predict

In [18]:
def predict_sentiment(model, sentence, tokenizer, max_len, device):
    model = model.eval()

    encoding = tokenizer.encode_plus(
        sentence,
        add_special_tokens=True,
        max_length=max_len,
        return_token_type_ids=False,
        pad_to_max_length=True,
        return_attention_mask=True,
        return_tensors='pt'
    )

    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)

    return outputs


def interpret_predictions(predictions):
    aspect_classes = ['SCREEN', 'CAMERA', 'FEATURES', 'BATTERY', 'PERFORMANCE', 'STORAGE', 'DESIGN', 'PRICE', 'GENERAL', 'SER&ACC']
    sentiment_labels = ['Positive', 'Negative', 'Neutral', 'None']

    results = {}

    for i, aspect in enumerate(aspect_classes):
        aspect_sentiments = predictions[0, i].cpu().numpy()
        sentiment_index = aspect_sentiments.argmax()
        results[aspect] = sentiment_labels[sentiment_index]

    return results

# 05. Thực hiện predict trên toàn bộ dữ liệu kiểm thử (testset)

In [19]:
test_data = pd.read_csv(test_path)
test_data.head()

,index,comment,n_star,date_time,label
0,0,"Điện thoải ổn. Facelock cực nhanh, vân tay ôk ...",5,5/2/2020,{SCREEN#Positive};{FEATURES#Positive};{PERFORM...
1,1,"Mình mới mua vivo91c. Tải ứng dụng ,games nh...",5,14/05/2019,{FEATURES#Negative};{PERFORMANCE#Positive};{SE...
2,2,Xấu đẹp gì ko biết nhưng rất ưng TGdđ phục vụ ...,5,26/03/2020,{DESIGN#Neutral};{SER&ACC#Positive};
3,3,Màn hình hơi lác khi chơi game. Game nặng thì ...,4,4/6/2019,{PERFORMANCE#Negative};{DESIGN#Negative};{OTHE...
4,4,"Nói chung máy đẹp với màn amoled, ổn trong tầm...",4,12/5/2020,{SCREEN#Positive};{BATTERY#Negative};{DESIGN#P...


In [20]:
max_len = 256
idx = 20
sentence_test = test_data.loc[idx, 'comment']
true_label = test_data.loc[idx, 'label']
print(sentence_test)
print(true_label)

predictions = predict_sentiment(model, sentence_test, tokenizer, max_len, device)
results = interpret_predictions(predictions)

print('Aspect Sentiment Prediction:')
for aspect, sentiment in results.items():
    print(f'{aspect}: {sentiment}')

Mọi thứ ok.mình thấy pin trâu chứ có sao đâu.mua đc 2 tuần rồi chỉ có cảm biến vân tay ko nhạy như mong đợi thôi.
{FEATURES#Negative};{BATTERY#Positive};{GENERAL#Positive};
Aspect Sentiment Prediction:
SCREEN: None
CAMERA: None
FEATURES: Negative
BATTERY: Positive
PERFORMANCE: None
STORAGE: None
DESIGN: None
PRICE: None
GENERAL: Positive
SER&ACC: None


In [21]:
predictions_on_test = []

for i in range(len(test_data)):
    print(i)
    sentence_on_test = test_data['comment'][i]
    prediction_on_test = predict_sentiment(model, sentence_on_test, tokenizer, max_len, device)
    results = interpret_predictions(prediction_on_test)
    predictions_on_test.append(results)
    clear_output()

print('All done!')


All done!


In [22]:
true_labels = []

for i in range(len(test_data)):
    outcome = {}
    label = test_data['label'][i]
    label = label.replace('{', '')
    label = label.replace('}', '')
    labels = label.split(';')
    for l in labels:
        try:
            text = l.split('#')
            outcome[text[0]] = text[1]
        except:
            pass
    true_labels.append(outcome)

# 06. Chuẩn hóa nhãn đầu ra để thực hiện đánh giá hiệu suất mô hình

In [23]:
def full_label(true_label, prediction):
    '''true_label: dict
     prediction: dict '''
    for key, value in prediction.items():
        if key not in true_label:
            true_label[key] = 'None'
    return true_label

In [24]:
# mục tiêu là đưa danh sách các nhãn thực tế về dạng đánh giá 10 class như trong dự đoán
true_full_labels = []
for idx in range(len(predictions_on_test)):
    true_label = true_labels[idx]
    prediction = predictions_on_test[idx]
    true_full_labels.append(full_label(true_label, prediction))

In [25]:
# data thỉnh thoảng bị sai -..-
# bằng chứng là SER&ACC trong câu này phải là Positive nhưng nhãn là None
# hoặc GENERAL nên là Positive thì nhãn lại là None

print(predictions_on_test[6])
print(true_labels[6])
print(test_data.loc[6, 'comment'])
for key, value in true_labels[6].items():
    print('true: ',key, value)
    print('predicted: ', key, predictions_on_test[6][key])

{'SCREEN': 'None', 'CAMERA': 'None', 'FEATURES': 'None', 'BATTERY': 'None', 'PERFORMANCE': 'Positive', 'STORAGE': 'None', 'DESIGN': 'None', 'PRICE': 'None', 'GENERAL': 'Positive', 'SER&ACC': 'Positive'}
{'SCREEN': 'None', 'CAMERA': 'None', 'FEATURES': 'None', 'BATTERY': 'None', 'PERFORMANCE': 'None', 'STORAGE': 'None', 'DESIGN': 'None', 'PRICE': 'None', 'GENERAL': 'None', 'SER&ACC': 'None'}
Thật tuyệt máy qua mượt túi thích  như thế giới Di Động có chỗ giá khác túi không hiểu đánh giá 5sao cho thế giới Di Động phương Phú Châu 5sao nhân viên OK
true:  SCREEN None
predicted:  SCREEN None
true:  CAMERA None
predicted:  CAMERA None
true:  FEATURES None
predicted:  FEATURES None
true:  BATTERY None
predicted:  BATTERY None
true:  PERFORMANCE None
predicted:  PERFORMANCE Positive
true:  STORAGE None
predicted:  STORAGE None
true:  DESIGN None
predicted:  DESIGN None
true:  PRICE None
predicted:  PRICE None
true:  GENERAL None
predicted:  GENERAL Positive
true:  SER&ACC None
predicted:  SER&A

# 07. Đánh giá hiệu suất tổng thể của mô hình

In [26]:
true_ = []
predicted_ = []

for idx in range(len(predictions_on_test)):
    for key, value in predictions_on_test[idx].items():
        if true_full_labels[idx][key] == 'None' and value == 'None':
            true_.append(0)
            predicted_.append(0)
        elif true_full_labels[idx][key] != 'None' and true_full_labels[idx][key] == value:
            true_.append(1)
            predicted_.append(1)
        elif true_full_labels[idx][key] == 'None' and value != 'None':
            true_.append(0)
            predicted_.append(1)
        elif true_full_labels[idx][key] != 'None' and value == 'None':
            true_.append(1)
            predicted_.append(0)

accuracy = accuracy_score(true_, predicted_)
precision = precision_score(true_, predicted_)
recall = recall_score(true_, predicted_)
f1 = f1_score(true_, predicted_)

print(f'Accuracy = {accuracy}')
print(f'Precision = {precision}')
print(f'Recall = {recall}')
print(f'f1 = {f1}')

Accuracy = 0.9321680267062314
Precision = 0.8765391561319981
Recall = 0.8824793388429752
f1 = 0.8794992175273866


# 08. Đánh giá hiệu suất nhận diện khía cạnh

### Accuracy, Precision, Recall, F1-score trên toàn bộ khía cạnh

In [27]:
true_ = []
predicted_ = []

for idx in range(len(predictions_on_test)):
    for key, value in predictions_on_test[idx].items():
        if true_full_labels[idx] == 'None' and value == 'None':
            true_.append(0)
            predicted_.append(0)
        elif true_full_labels[idx][key] != 'None' and value != 'None':
            true_.append(1)
            predicted_.append(1)
        elif true_full_labels[idx][key] == 'None' and value != 'None':
            true_.append(0)
            predicted_.append(1)
        elif true_full_labels[idx][key] != 'None' and value == 'None':
            true_.append(1)
            predicted_.append(0)

print(f'Accuracy = {accuracy_score(true_, predicted_)}')
print(f'Precision = {precision_score(true_, predicted_)}')
print(f'Recall = {recall_score(true_, predicted_)}')
print(f'F1-score = {f1_score(true_, predicted_)}')

Accuracy = 0.8042547497993042
Precision = 0.8888067425698655
Recall = 0.894227908360607
F1-score = 0.8915090841675937


### đánh giá hiệu suất khía cạnh trên class SCREEN

In [28]:
true_ = []
predicted_ = []

for idx in range(len(true_full_labels)):
    key = 'SCREEN'
    if true_full_labels[idx][key] == 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(0)
        predicted_.append(0)
    elif true_full_labels[idx][key] != 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(1)
        predicted_.append(1)
    elif true_full_labels[idx][key] == 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(0)
        predicted_.append(1)
    elif true_full_labels[idx][key] != 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(1)
        predicted_.append(0)

accuracy = accuracy_score(true_, predicted_)
precision = precision_score(true_, predicted_)
recall = recall_score(true_, predicted_)
f1 = f1_score(true_, predicted_)

print(f'Accuracy = {accuracy}')
print(f'Precision = {precision}')
print(f'Recall = {recall}')
print(f'f1 = {f1}')

Accuracy = 0.9532374100719424
Precision = 0.8947368421052632
Recall = 0.6951672862453532
f1 = 0.7824267782426778


### đánh giá hiệu suất khía cạnh trên class FEATURES

In [29]:
true_ = []
predicted_ = []

for idx in range(len(true_full_labels)):
    key = 'FEATURES'
    if true_full_labels[idx][key] == 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(0)
        predicted_.append(0)
    elif true_full_labels[idx][key] != 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(1)
        predicted_.append(1)
    elif true_full_labels[idx][key] == 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(0)
        predicted_.append(1)
    elif true_full_labels[idx][key] != 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(1)
        predicted_.append(0)

accuracy = accuracy_score(true_, predicted_)
precision = precision_score(true_, predicted_)
recall = recall_score(true_, predicted_)
f1 = f1_score(true_, predicted_)

print(f'Accuracy = {accuracy}')
print(f'Precision = {precision}')
print(f'Recall = {recall}')
print(f'f1 = {f1}')

Accuracy = 0.9087230215827338
Precision = 0.8207070707070707
Recall = 0.9142053445850914
f1 = 0.8649367930805056


### đánh giá hiệu suất khía cạnh trên class PERFORMANCE

In [30]:
true_ = []
predicted_ = []

for idx in range(len(true_full_labels)):
    key = 'PERFORMANCE'
    if true_full_labels[idx][key] == 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(0)
        predicted_.append(0)
    elif true_full_labels[idx][key] != 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(1)
        predicted_.append(1)
    elif true_full_labels[idx][key] == 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(0)
        predicted_.append(1)
    elif true_full_labels[idx][key] != 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(1)
        predicted_.append(0)

accuracy = accuracy_score(true_, predicted_)
precision = precision_score(true_, predicted_)
recall = recall_score(true_, predicted_)
f1 = f1_score(true_, predicted_)

print(f'Accuracy = {accuracy}')
print(f'Precision = {precision}')
print(f'Recall = {recall}')
print(f'f1 = {f1}')

Accuracy = 0.89568345323741
Precision = 0.9174067495559503
Recall = 0.8813993174061433
f1 = 0.8990426457789382


### đánh giá hiệu suất khía cạnh trên class GENERAL

In [31]:
true_ = []
predicted_ = []

for idx in range(len(true_full_labels)):
    key = 'GENERAL'
    if true_full_labels[idx][key] == 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(0)
        predicted_.append(0)
    elif true_full_labels[idx][key] != 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(1)
        predicted_.append(1)
    elif true_full_labels[idx][key] == 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(0)
        predicted_.append(1)
    elif true_full_labels[idx][key] != 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(1)
        predicted_.append(0)

accuracy = accuracy_score(true_, predicted_)
precision = precision_score(true_, predicted_)
recall = recall_score(true_, predicted_)
f1 = f1_score(true_, predicted_)

print(f'Accuracy = {accuracy}')
print(f'Precision = {precision}')
print(f'Recall = {recall}')
print(f'f1 = {f1}')

Accuracy = 0.8552158273381295
Precision = 0.9051262433052792
Recall = 0.8566256335988415
f1 = 0.8802083333333334


### đánh giá hiệu suất khía cạnh trên class BATTERY

In [32]:
true_ = []
predicted_ = []

for idx in range(len(true_full_labels)):
    key = 'BATTERY'
    if true_full_labels[idx][key] == 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(0)
        predicted_.append(0)
    elif true_full_labels[idx][key] != 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(1)
        predicted_.append(1)
    elif true_full_labels[idx][key] == 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(0)
        predicted_.append(1)
    elif true_full_labels[idx][key] != 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(1)
        predicted_.append(0)

accuracy = accuracy_score(true_, predicted_)
precision = precision_score(true_, predicted_)
recall = recall_score(true_, predicted_)
f1 = f1_score(true_, predicted_)

print(f'Accuracy = {accuracy}')
print(f'Precision = {precision}')
print(f'Recall = {recall}')
print(f'f1 = {f1}')

Accuracy = 0.9680755395683454
Precision = 0.9573229873908826
Recall = 0.9733727810650887
f1 = 0.965281173594132


### đánh giá hiệu suất khía cạnh trên class STORAGE

In [33]:
true_ = []
predicted_ = []

for idx in range(len(true_full_labels)):
    key = 'STORAGE'
    if true_full_labels[idx][key] == 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(0)
        predicted_.append(0)
    elif true_full_labels[idx][key] != 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(1)
        predicted_.append(1)
    elif true_full_labels[idx][key] == 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(0)
        predicted_.append(1)
    elif true_full_labels[idx][key] != 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(1)
        predicted_.append(0)

accuracy = accuracy_score(true_, predicted_)
precision = precision_score(true_, predicted_)
recall = recall_score(true_, predicted_)
f1 = f1_score(true_, predicted_)

print(f'Accuracy = {accuracy}')
print(f'Precision = {precision}')
print(f'Recall = {recall}')
print(f'f1 = {f1}')

Accuracy = 0.9923561151079137
Precision = 0.7083333333333334
Recall = 0.6296296296296297
f1 = 0.6666666666666666


### đánh giá hiệu suất khía cạnh trên class DESIGN

In [34]:
true_ = []
predicted_ = []

for idx in range(len(true_full_labels)):
    key = 'DESIGN'
    if true_full_labels[idx][key] == 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(0)
        predicted_.append(0)
    elif true_full_labels[idx][key] != 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(1)
        predicted_.append(1)
    elif true_full_labels[idx][key] == 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(0)
        predicted_.append(1)
    elif true_full_labels[idx][key] != 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(1)
        predicted_.append(0)

accuracy = accuracy_score(true_, predicted_)
precision = precision_score(true_, predicted_)
recall = recall_score(true_, predicted_)
f1 = f1_score(true_, predicted_)

print(f'Accuracy = {accuracy}')
print(f'Precision = {precision}')
print(f'Recall = {recall}')
print(f'f1 = {f1}')

Accuracy = 0.9410971223021583
Precision = 0.8027210884353742
Recall = 0.8894472361809045
f1 = 0.8438617401668653


### đánh giá hiệu suất khía cạnh trên class PRICE

In [35]:
true_ = []
predicted_ = []

for idx in range(len(true_full_labels)):
    key = 'PRICE'
    if true_full_labels[idx][key] == 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(0)
        predicted_.append(0)
    elif true_full_labels[idx][key] != 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(1)
        predicted_.append(1)
    elif true_full_labels[idx][key] == 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(0)
        predicted_.append(1)
    elif true_full_labels[idx][key] != 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(1)
        predicted_.append(0)

accuracy = accuracy_score(true_, predicted_)
precision = precision_score(true_, predicted_)
recall = recall_score(true_, predicted_)
f1 = f1_score(true_, predicted_)

print(f'Accuracy = {accuracy}')
print(f'Precision = {precision}')
print(f'Recall = {recall}')
print(f'f1 = {f1}')

Accuracy = 0.9514388489208633
Precision = 0.8835274542429284
Recall = 0.9332161687170475
f1 = 0.9076923076923077


### đánh giá hiệu suất khía cạnh trên class SER&ACC

In [36]:
true_ = []
predicted_ = []

for idx in range(len(true_full_labels)):
    key = 'SER&ACC'
    if true_full_labels[idx][key] == 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(0)
        predicted_.append(0)
    elif true_full_labels[idx][key] != 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(1)
        predicted_.append(1)
    elif true_full_labels[idx][key] == 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(0)
        predicted_.append(1)
    elif true_full_labels[idx][key] != 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(1)
        predicted_.append(0)

accuracy = accuracy_score(true_, predicted_)
precision = precision_score(true_, predicted_)
recall = recall_score(true_, predicted_)
f1 = f1_score(true_, predicted_)

print(f'Accuracy = {accuracy}')
print(f'Precision = {precision}')
print(f'Recall = {recall}')
print(f'f1 = {f1}')

Accuracy = 0.9019784172661871
Precision = 0.7971473851030111
Recall = 0.8482293423271501
f1 = 0.8218954248366013


### đánh giá hiệu suất khía cạnh trên class CAMERA

In [37]:
true_ = []
predicted_ = []

for idx in range(len(true_full_labels)):
    key = 'CAMERA'
    if true_full_labels[idx][key] == 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(0)
        predicted_.append(0)
    elif true_full_labels[idx][key] != 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(1)
        predicted_.append(1)
    elif true_full_labels[idx][key] == 'None' and predictions_on_test[idx][key] != 'None':
        true_.append(0)
        predicted_.append(1)
    elif true_full_labels[idx][key] != 'None' and predictions_on_test[idx][key] == 'None':
        true_.append(1)
        predicted_.append(0)

accuracy = accuracy_score(true_, predicted_)
precision = precision_score(true_, predicted_)
recall = recall_score(true_, predicted_)
f1 = f1_score(true_, predicted_)

print(f'Accuracy = {accuracy}')
print(f'Precision = {precision}')
print(f'Recall = {recall}')
print(f'f1 = {f1}')

Accuracy = 0.9743705035971223
Precision = 0.9417637271214643
Recall = 0.9625850340136054
f1 = 0.9520605550883096


# 09. Đánh giá hiệu suất nhận diện tình cảm (đa nhãn)

### Accuracy trên toàn bộ data kiểm thử
acc = corrects/(all predictions)

In [38]:
true_ = []
predicted_ = []
accuracy = []

for idx in range(len(true_full_labels)):
    for key, value in true_full_labels[idx].items():
        if value == 'None':
            continue
        elif predictions_on_test[idx][key] != value:
            accuracy.append(0)
        elif predictions_on_test[idx][key] == value:
            accuracy.append(1)

print(f'Accuracy = {sum(accuracy)/len(accuracy)}')

Accuracy = 0.7942576614102945


### Precisio, Recall, F1-score tình cảm trên toàn bộ data kiểm thử

In [39]:
from sklearn.metrics import classification_report


y_true = []
y_pred = []

for idx in range(len(true_labels)):
    for key, value in true_labels[idx].items():
        y_true.append(key + "_" + value)
        y_pred.append(key + "_" + predictions_on_test[idx][key])


print(f'Classification report:')
print(classification_report(y_true, y_pred, digits=4))

Classification report:
                      precision    recall  f1-score   support

    BATTERY_Negative     0.8660    0.8777    0.8718       368
     BATTERY_Neutral     0.4324    0.5217    0.4729        92
        BATTERY_None     0.9774    0.9636    0.9705      1210
    BATTERY_Positive     0.9086    0.8971    0.9028       554
     CAMERA_Negative     0.8497    0.7602    0.8025       171
      CAMERA_Neutral     0.5000    0.6338    0.5590        71
         CAMERA_None     0.9864    0.9786    0.9825      1636
     CAMERA_Positive     0.9078    0.9393    0.9233       346
     DESIGN_Negative     0.4242    0.5833    0.4912        96
      DESIGN_Neutral     0.2000    0.1429    0.1667        28
         DESIGN_None     0.9753    0.9524    0.9637      1826
     DESIGN_Positive     0.8547    0.9015    0.8774       274
   FEATURES_Negative     0.7514    0.8758    0.8089       459
    FEATURES_Neutral     0.3016    0.3654    0.3304        52
       FEATURES_None     0.9574    0.9061    0

In [40]:
precision_micro = precision_score(y_true, y_pred, average='micro')
recall_micro = recall_score(y_true, y_pred, average='micro')
f1_micro = f1_score(y_true, y_pred, average='micro')

precision_macro = precision_score(y_true, y_pred, average='macro')
recall_macro = recall_score(y_true, y_pred, average='macro')
f1_macro = f1_score(y_true, y_pred, average='macro')

precision_weighted = precision_score(y_true, y_pred, average='weighted')
recall_weighted = recall_score(y_true, y_pred, average='weighted')
f1_weighted = f1_score(y_true, y_pred, average='weighted')

print("\nOverall Precision, Recall, F1-Score:")
print(f"Micro Precision: {precision_micro:.4f}")
print(f"Micro Recall: {recall_micro:.4f}")
print(f"Micro F1-Score: {f1_micro:.4f}")
print(f"Macro Precision: {precision_macro:.4f}")
print(f"Macro Recall: {recall_macro:.4f}")
print(f"Macro F1-Score: {f1_macro:.4f}")
print(f"Weighted Precision: {precision_weighted:.4f}")
print(f"Weighted Recall: {recall_weighted:.4f}")
print(f"Weighted F1-Score: {f1_weighted:.4f}")


Overall Precision, Recall, F1-Score:
Micro Precision: 0.9040
Micro Recall: 0.9040
Micro F1-Score: 0.9040
Macro Precision: 0.6977
Macro Recall: 0.6936
Macro F1-Score: 0.6904
Weighted Precision: 0.9068
Weighted Recall: 0.9040
Weighted F1-Score: 0.9042


### Đánh giá hiệu suất nhận diện tình cảm của khía cạnh SCREEN

In [41]:
y_true = []
y_pred = []

key = 'SCREEN'
for idx in range(len(true_full_labels)):
    y_true.append(true_full_labels[idx][key])
    y_pred.append(predictions_on_test[idx][key])

report = classification_report(y_true, y_pred)
print(report)

              precision    recall  f1-score   support

    Negative       0.79      0.54      0.64       116
     Neutral       0.40      0.12      0.18        17
        None       0.96      0.99      0.97      1955
    Positive       0.87      0.79      0.83       136

    accuracy                           0.95      2224
   macro avg       0.75      0.61      0.66      2224
weighted avg       0.94      0.95      0.94      2224



### Đánh giá hiệu suất nhận diện tình cảm của khía cạnh FEATURES

In [42]:
y_true = []
y_pred = []

key = 'FEATURES'
for idx in range(len(true_full_labels)):
    y_true.append(true_full_labels[idx][key])
    y_pred.append(predictions_on_test[idx][key])

report = classification_report(y_true, y_pred)
print(report)

              precision    recall  f1-score   support

    Negative       0.75      0.88      0.81       459
     Neutral       0.30      0.37      0.33        52
        None       0.96      0.91      0.93      1513
    Positive       0.75      0.73      0.74       200

    accuracy                           0.87      2224
   macro avg       0.69      0.72      0.70      2224
weighted avg       0.88      0.87      0.87      2224



### Đánh giá hiệu suất nhận diện tình cảm của khía cạnh PERFORMANCE

In [43]:
y_true = []
y_pred = []

key = 'PERFORMANCE'
for idx in range(len(true_full_labels)):
    y_true.append(true_full_labels[idx][key])
    y_pred.append(predictions_on_test[idx][key])

report = classification_report(y_true, y_pred)
print(report)

              precision    recall  f1-score   support

    Negative       0.83      0.73      0.78       454
     Neutral       0.43      0.40      0.41       116
        None       0.87      0.91      0.89      1052
    Positive       0.86      0.88      0.87       602

    accuracy                           0.84      2224
   macro avg       0.75      0.73      0.74      2224
weighted avg       0.84      0.84      0.84      2224



### Đánh giá hiệu suất nhận diện tình cảm của khía cạnh GENERAL

In [44]:
y_true = []
y_pred = []

key = 'GENERAL'
for idx in range(len(true_full_labels)):
    y_true.append(true_full_labels[idx][key])
    y_pred.append(predictions_on_test[idx][key])

report = classification_report(y_true, y_pred)
print(report)

              precision    recall  f1-score   support

    Negative       0.82      0.59      0.69       294
     Neutral       0.66      0.51      0.57        83
        None       0.78      0.85      0.82       843
    Positive       0.89      0.91      0.90      1004

    accuracy                           0.83      2224
   macro avg       0.79      0.71      0.74      2224
weighted avg       0.83      0.83      0.83      2224



### Đánh giá hiệu suất nhận diện tình cảm của khía cạnh CAMERA

In [45]:
y_true = []
y_pred = []

key = 'CAMERA'
for idx in range(len(true_full_labels)):
    y_true.append(true_full_labels[idx][key])
    y_pred.append(predictions_on_test[idx][key])

report = classification_report(y_true, y_pred)
print(report)

              precision    recall  f1-score   support

    Negative       0.85      0.76      0.80       171
     Neutral       0.50      0.63      0.56        71
        None       0.99      0.98      0.98      1636
    Positive       0.91      0.94      0.92       346

    accuracy                           0.94      2224
   macro avg       0.81      0.83      0.82      2224
weighted avg       0.95      0.94      0.95      2224



### Đánh giá hiệu suất nhận diện tình cảm của khía cạnh BATTERY

In [46]:
y_true = []
y_pred = []

key = 'BATTERY'
for idx in range(len(true_full_labels)):
    y_true.append(true_full_labels[idx][key])
    y_pred.append(predictions_on_test[idx][key])

report = classification_report(y_true, y_pred)
print(report)

              precision    recall  f1-score   support

    Negative       0.87      0.88      0.87       368
     Neutral       0.43      0.52      0.47        92
        None       0.98      0.96      0.97      1210
    Positive       0.91      0.90      0.90       554

    accuracy                           0.91      2224
   macro avg       0.80      0.82      0.80      2224
weighted avg       0.92      0.91      0.92      2224



### Đánh giá hiệu suất nhận diện tình cảm của khía cạnh STORAGE

In [47]:
y_true = []
y_pred = []

key = 'STORAGE'
for idx in range(len(true_full_labels)):
    y_true.append(true_full_labels[idx][key])
    y_pred.append(predictions_on_test[idx][key])

report = classification_report(y_true, y_pred)
print(report)

              precision    recall  f1-score   support

    Negative       0.00      0.00      0.00         6
     Neutral       0.00      0.00      0.00         3
        None       1.00      1.00      1.00      2197
    Positive       0.60      0.67      0.63        18

    accuracy                           0.99      2224
   macro avg       0.40      0.42      0.41      2224
weighted avg       0.99      0.99      0.99      2224



### Đánh giá hiệu suất nhận diện tình cảm của khía cạnh DESIGN

In [48]:
y_true = []
y_pred = []

key = 'DESIGN'
for idx in range(len(true_full_labels)):
    y_true.append(true_full_labels[idx][key])
    y_pred.append(predictions_on_test[idx][key])

report = classification_report(y_true, y_pred)
print(report)

              precision    recall  f1-score   support

    Negative       0.42      0.58      0.49        96
     Neutral       0.20      0.14      0.17        28
        None       0.98      0.95      0.96      1826
    Positive       0.85      0.90      0.88       274

    accuracy                           0.92      2224
   macro avg       0.61      0.65      0.62      2224
weighted avg       0.93      0.92      0.92      2224



### Đánh giá hiệu suất nhận diện tình cảm của khía cạnh PRICE

In [49]:
y_true = []
y_pred = []

key = 'PRICE'
for idx in range(len(true_full_labels)):
    y_true.append(true_full_labels[idx][key])
    y_pred.append(predictions_on_test[idx][key])

report = classification_report(y_true, y_pred)
print(report)

              precision    recall  f1-score   support

    Negative       0.51      0.61      0.55        79
     Neutral       0.81      0.69      0.74       328
        None       0.98      0.96      0.97      1655
    Positive       0.61      0.86      0.72       162

    accuracy                           0.90      2224
   macro avg       0.73      0.78      0.75      2224
weighted avg       0.91      0.90      0.90      2224



### Đánh giá hiệu suất nhận diện tình cảm của khía cạnh SER&ACC

In [50]:
y_true = []
y_pred = []

key = 'SER&ACC'
for idx in range(len(true_full_labels)):
    y_true.append(true_full_labels[idx][key])
    y_pred.append(predictions_on_test[idx][key])

report = classification_report(y_true, y_pred)
print(report)

              precision    recall  f1-score   support

    Negative       0.55      0.61      0.58       167
     Neutral       0.27      0.30      0.28        27
        None       0.94      0.92      0.93      1631
    Positive       0.85      0.89      0.87       399

    accuracy                           0.88      2224
   macro avg       0.65      0.68      0.67      2224
weighted avg       0.89      0.88      0.89      2224

